In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [6]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='US/Eastern')
feature_store = get_feature_store()

2025-05-08 22:24:47,699 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 22:24:47,713 INFO: Initializing external client
2025-05-08 22:24:47,715 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-08 22:24:48,965 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682


In [8]:
feature_group = feature_store.get_feature_group(
    name=config.FEATURE_GROUP_NAME,
    version=config.FEATURE_GROUP_VERSION
)
feature_store.create_feature_view(
        name=config.FEATURE_VIEW_NAME,
        version=config.FEATURE_VIEW_VERSION,
        query=feature_group.select_all(),
    )

Feature view created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/1213682/fs/1202282/fv/time_series_hourly_feature_view/version/1


In [9]:
# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")

feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)
ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.start_hour.between(fetch_data_from, fetch_data_to)]

Fetching data from 2025-04-09 22:24:47.696430-04:00 to 2025-05-08 21:24:47.696430-04:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.55s) 


In [10]:
ts_data.sort_values(["start_station_id", "start_hour"]).reset_index(drop=True)

,start_hour,start_station_id,rides
0,2025-04-10 03:00:00+00:00,5308.04,1
1,2025-04-10 04:00:00+00:00,5308.04,0
2,2025-04-10 05:00:00+00:00,5308.04,0
3,2025-04-10 06:00:00+00:00,5308.04,2
4,2025-04-10 07:00:00+00:00,5308.04,7
...,...,...,...
2080,2025-05-08 21:00:00+00:00,6230.02,3
2081,2025-05-08 22:00:00+00:00,6230.02,8
2082,2025-05-08 23:00:00+00:00,6230.02,11
2083,2025-05-09 00:00:00+00:00,6230.02,7


In [11]:
ts_data["start_hour"] = ts_data["start_hour"].dt.tz_convert('US/Eastern')


In [12]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [13]:
features["start_hour"] = features["start_hour"].dt.tz_localize('US/Eastern', nonexistent = 'shift_forward', ambiguous = 'NaT')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Columns: 674 entries, rides_t-672 to start_hour
dtypes: datetime64[ns, US/Eastern](1), float64(1), int64(672)
memory usage: 15.9 KB


In [14]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,start_station_id,start_hour
0,28,9,9,12,8,0,9,29,14,10,...,1,2,5,10,16,1,28,1,5308.04,2025-04-16 04:00:00-04:00
1,6,5,1,3,4,0,2,0,0,2,...,3,0,14,3,0,14,11,4,6230.02,2025-05-04 04:00:00-04:00
2,1,6,28,16,4,14,16,1,0,10,...,12,0,11,11,0,2,1,2,5872.10,2025-04-23 01:00:00-04:00


In [16]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='US/Eastern')
features = load_batch_of_features_from_store(current_date)

2025-05-08 22:30:53,074 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-08 22:30:53,098 INFO: Initializing external client
2025-05-08 22:30:53,098 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-05-08 22:30:54,295 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682
Fetching data from 2025-04-09 22:30:53.074287-04:00 to 2025-05-08 21:30:53.074287-04:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.90s) 


In [17]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,start_station_id,start_hour
0,1,0,0,2,7,13,7,9,9,5,...,30,29,17,15,12,8,4,2,5308.04,2025-05-08 03:00:00
1,0,0,5,2,6,4,8,2,10,7,...,21,11,11,9,6,1,3,0,5872.10,2025-05-08 03:00:00
2,0,1,1,5,5,20,9,7,10,9,...,15,9,8,4,7,0,0,1,6230.02,2025-05-08 03:00:00


In [19]:
features["start_hour"] = features["start_hour"].dt.tz_localize('US/Eastern', nonexistent = 'shift_forward', ambiguous = 'NaT')
features.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Columns: 674 entries, rides_t-672 to start_hour
dtypes: datetime64[ns, US/Eastern](1), float64(1), int64(672)
memory usage: 15.9 KB


In [21]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-05-09 01:21:04,454 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-09 01:21:04,508 INFO: Initializing external client
2025-05-09 01:21:04,508 INFO: Base URL: https://c.app.hopsworks.ai:443




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (4.2) by running 'pip install hopsworks==4.2.*'


2025-05-09 01:21:05,954 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1213682

In [22]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [23]:
predictions

,start_station_id,predicted_demand
0,5308.04,2.0
1,5872.10,1.0
2,6230.02,0.0
